In [ ]:
"""
Weekend Support Agent (Tech Lead Version)
-----------------------------------------

This agent retrieves knowledge from Confluence and also allows
optional "agent instructions" to be pulled from a specific Confluence
page. This page acts as a system prompt, enabling departments to 
control agent behavior without touching code.

Flow:
    START → LoadPromptPage → RetrieveDocs → Summarize → Action → Final → END
"""

from __future__ import annotations
import os
from pathlib import Path
from typing import TypedDict
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import ConfluenceLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS

from langchain_core.runnables import RunnableLambda
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

# -----------------------------------------------------------------------------
# 🔑 Environment Setup
# -----------------------------------------------------------------------------
ROOT_DIR = Path(__file__).resolve().parents[1]
load_dotenv(ROOT_DIR / ".env")

if not os.getenv("OPENAI_API_KEY"):
    raise RuntimeError("OPENAI_API_KEY is not defined in .env")

OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o")
CONFLUENCE_URL = os.getenv("CONFLUENCE_URL")
CONFLUENCE_USER = os.getenv("CONFLUENCE_USER")
CONFLUENCE_API_TOKEN = os.getenv("CONFLUENCE_API_TOKEN")
CONFLUENCE_SPACE_KEY = os.getenv("CONFLUENCE_SPACE_KEY", "IT")
CONFLUENCE_PROMPT_PAGE_ID = os.getenv("CONFLUENCE_PROMPT_PAGE_ID", None)  # Optional page ID for instructions

llm = ChatOpenAI(model=OPENAI_MODEL)

# -----------------------------------------------------------------------------
# 🗂️ State Definition
# -----------------------------------------------------------------------------
class State(TypedDict):
    task: str
    lang: str
    prompt_instructions: str  # Optional system prompt loaded from Confluence
    docs: str
    summary: str
    action: str
    final_summary: str

# -----------------------------------------------------------------------------
# 🌐 Helpers
# -----------------------------------------------------------------------------
def lang_prefix(lang: str, es: str, en: str) -> str:
    return es if lang == "es" else en

def _ctx(state: State) -> str:
    return f"Task: {state.get('task','')}\n"

# -----------------------------------------------------------------------------
# 📘 Confluence Knowledge Base
# -----------------------------------------------------------------------------
def build_retriever() -> FAISS:
    """Load Confluence documents into FAISS retriever."""
    loader = ConfluenceLoader(
        url=CONFLUENCE_URL,
        username=CONFLUENCE_USER,
        api_key=CONFLUENCE_API_TOKEN,
        space_key=CONFLUENCE_SPACE_KEY,
        include_attachments=False,
        limit=100,
    )
    docs = loader.load()

    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = splitter.split_documents(docs)

    embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
    db = FAISS.from_documents(chunks, embeddings)
    return db.as_retriever(search_kwargs={"k": 3})

retriever = build_retriever()

def load_prompt_page() -> str:
    """Load optional prompt instructions from a dedicated Confluence page."""
    if not CONFLUENCE_PROMPT_PAGE_ID:
        return ""
    loader = ConfluenceLoader(
        url=CONFLUENCE_URL,
        username=CONFLUENCE_USER,
        api_key=CONFLUENCE_API_TOKEN,
        page_ids=[CONFLUENCE_PROMPT_PAGE_ID],  # load only this page
    )
    docs = loader.load()
    return docs[0].page_content if docs else ""

# -----------------------------------------------------------------------------
# 🤖 Agent Nodes
# -----------------------------------------------------------------------------
def prompt_instructions_agent(state: State) -> State:
    """Load optional system instructions from Confluence."""
    state["prompt_instructions"] = load_prompt_page()
    return state

def retrieve_docs(state: State) -> State:
    """Retrieve relevant docs from Confluence."""
    query = state.get("task", "")
    results = retriever.get_relevant_documents(query)
    state["docs"] = "\n\n".join([r.page_content for r in results])
    return state

def summarize_docs(state: State) -> State:
    """Summarize retrieved docs."""
    prompt = f"""{lang_prefix(state['lang'],
    "Explica en lenguaje sencillo lo encontrado en Confluence.",
    "Summarize the retrieved Confluence documentation in plain language.")}

{_ctx(state)}

⚙️ Agent Instructions (if any):
{state['prompt_instructions']}

📚 Docs:
{state['docs']}
"""
    result = llm.invoke(prompt)
    state["summary"] = result.content
    return state

def action_step(state: State) -> State:
    """Suggest one practical action."""
    prompt = f"""{lang_prefix(state['lang'],
    "Sugiere una acción práctica (≤15 min).",
    "Suggest one practical action (≤15 min).")}

{_ctx(state)}
Summary:
{state['summary']}
"""
    result = llm.invoke(prompt)
    state["action"] = result.content
    return state

def final_summary(state: State) -> State:
    """Stitch everything together into a final response."""
    prompt = f"""{lang_prefix(state['lang'],
    "📋 Respuesta Final de Soporte",
    "📋 Final Support Answer")}

{_ctx(state)}

⚙️ Agent Instructions:
{state['prompt_instructions']}

📚 Docs:
{state['docs']}

💡 Explanation:
{state['summary']}

⚡ Suggested Action:
{state['action']}
"""
    result = llm.invoke(prompt)
    state["final_summary"] = result.content
    return state

# -----------------------------------------------------------------------------
# 🔗 Graph Assembly
# -----------------------------------------------------------------------------
builder = StateGraph(State)
builder.add_node("PromptInstructions", RunnableLambda(prompt_instructions_agent))
builder.add_node("RetrieveDocs", RunnableLambda(retrieve_docs))
builder.add_node("Summarize", RunnableLambda(summarize_docs))
builder.add_node("Action", RunnableLambda(action_step))
builder.add_node("Final", RunnableLambda(final_summary))

builder.add_edge(START, "PromptInstructions")
builder.add_edge("PromptInstructions", "RetrieveDocs")
builder.add_edge("RetrieveDocs", "Summarize")
builder.add_edge("Summarize", "Action")
builder.add_edge("Action", "Final")
builder.add_edge("Final", END)

graph = builder.compile(checkpointer=MemorySaver())

# -----------------------------------------------------------------------------
# 🚀 Entry Point
# -----------------------------------------------------------------------------
def run_support(task: str, lang: str = "en") -> dict:
    """Run the Weekend Support pipeline."""
    state_in: State = {
        "task": task or "",
        "lang": lang or "en",
        "prompt_instructions": "",
        "docs": "",
        "summary": "",
        "action": "",
        "final_summary": "",
    }
    result: State = graph.invoke(state_in, config={"configurable": {"thread_id": "weekend-support"}})
    return {
        "prompt_instructions": result.get("prompt_instructions", ""),
        "docs": result.get("docs", ""),
        "summary": result.get("summary", ""),
        "action": result.get("action", ""),
        "final_summary": result.get("final_summary", ""),
    }
